<a href="https://colab.research.google.com/github/anitanadvikova/ScrabbleGAN_RU/blob/main/fid_344.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/anitanadvikova/ScrabbleGAN.git

Cloning into 'ScrabbleGAN'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 145 (delta 30), reused 20 (delta 20), pack-reused 101
Receiving objects: 100% (145/145), 194.51 KiB | 3.19 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd ScrabbleGAN/data

/content/ScrabbleGAN/data


In [6]:
!unzip /content/drive/MyDrive/RIMES.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: RIMES/training/www10957.png  
  inflating: __MACOSX/RIMES/training/._www10957.png  
  inflating: RIMES/training/pe12664.png  
  inflating: __MACOSX/RIMES/training/._pe12664.png  
  inflating: RIMES/training/az6827.png  
  inflating: __MACOSX/RIMES/training/._az6827.png  
  inflating: RIMES/training/dem29563.png  
  inflating: __MACOSX/RIMES/training/._dem29563.png  
  inflating: RIMES/training/af1178.png  
  inflating: __MACOSX/RIMES/training/._af1178.png  
  inflating: RIMES/training/auu1645.png  
  inflating: __MACOSX/RIMES/training/._auu1645.png  
  inflating: RIMES/training/dem18966.png  
  inflating: __MACOSX/RIMES/training/._dem18966.png  
  inflating: RIMES/training/iy190.png  
  inflating: __MACOSX/RIMES/training/._iy190.png  
  inflating: RIMES/training/dem30032.png  
  inflating: __MACOSX/RIMES/training/._dem30032.png  
  inflating: RIMES/training/dem5402.png  
  inflating: __MACOSX/RIMES/trainin

In [7]:
!python prepare_data.py

Processing Data:

Number of images = 17913
Number of unique characters = 108

Data processing completed


In [ ]:
%cd ..

/content/ScrabbleGAN


In [ ]:
#%rm -rf fid_calc

In [ ]:
!bash  python calculate_metrics.py -c '/content/ScrabbleGAN/weights/model_checkpoint_epoch_1.pth.tar' 

Generating and saving fake images
Model: ScrabbleGAN
Param count for Gs initialized parameters: 30869377
Param count for Ds initialized parameters: 36398401
Traceback (most recent call last):
  File "/content/ScrabbleGAN/calculate_metrics.py", line 77, in <module>
    calculate_fid(checkpoint_path, num_imgs)
  File "/content/ScrabbleGAN/calculate_metrics.py", line 40, in calculate_fid
    generator = ImgGenerator(checkpt_path=checkpt_path, config=config)
  File "/content/ScrabbleGAN/generate_images.py", line 40, in __init__
    self.model, _, _, _ = self.model_checkpoint.load(self.model, epoch=None, checkpoint_path=checkpt_path)
  File "/content/ScrabbleGAN/utils/training_utils.py", line 36, in load
    checkpoint = torch.load(load_filename, map_location=self.config.device)
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 777, in load
    with _open_zipfile_reader(opened_file) as opened_zipfile:
  File "/usr/local/lib/python3.9/dist-packages/torch/serializat

In [ ]:
!python -m pytorch_fid '/content/ScrabbleGAN/fid_calc/real' '/content/ScrabbleGAN/fid_calc/fake'

/usr/bin/python3: No module named pytorch_fid


In [ ]:
#!tar -xf /content/ScrabbleGAN/weights/model_checkpoint_epoch_1.pth.tar

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [ ]:
import torch

In [ ]:
torch.load('/content/ScrabbleGAN/weights/model_checkpoint_epoch_344.pth.tar')

RuntimeError: ignored

In [9]:
!pip install pytorch_fid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import cv2

In [17]:
r = cv2.imread('/content/ScrabbleGAN/data/RIMES/validation/test1.png')

In [16]:
r.shape

(50, 88, 3)

In [18]:
r.shape

(46, 158, 3)

In [ ]:
def calculate_fid(checkpt_path, num_images=25000):
    """
    :param checkpt_path: Path of the model checkpoint file to be used
    :param num_images: Number of images from real and fake images to be taken for consideration
    :return: fid score
    """
    config = Config
    path = './fid_calc/'
    fake_path = path + '/fake/'
    real_path = path + '/real/'
    # Create directories for saving fake and real images
    os.makedirs(fake_path, exist_ok=True)
    os.makedirs(real_path, exist_ok=True)

    # fake images
    print('Generating and saving fake images')
    generator = ImgGenerator(checkpt_path=checkpt_path, config=config)

    # save these fake images
    for idx, img in enumerate(tqdm(range(num_images))):
        img, _, word_labels = generator.generate(1)
        img = img_resize(img[0]*255)
        img = Image.fromarray(img).convert("RGB")
        img.save(f'{fake_path}/{idx}.png')

    # real images
    print('Sampling and saving real images')
    dataset = CustomDataset(config)
    # choose random images
    imgs_idx = np.random.choice(len(dataset), num_images)
    for idx, img_idx in enumerate(tqdm(imgs_idx)):
        w_id = dataset.idx_to_id[img_idx]
        # Get image
        _, img = dataset.word_data[w_id]
        img = img_resize(img)
        img = Image.fromarray(img).convert("RGB")
        img.save(f'{real_path}/{idx}.png')

    # calculate fid
    os.system(f'python -m pytorch_fid {real_path} {fake_path}')

In [19]:
import glob

In [64]:
!rm -rf temp_fake

In [65]:
!mkdir temp_fake

In [66]:
!rm -rf temp_real

In [67]:
!mkdir temp_real

In [23]:
images_fake = glob.glob('/content/drive/MyDrive/cyr_val/*')

In [24]:
fake_path = 'temp_fake'

In [26]:
import tqdm

In [32]:
import numpy as np

In [35]:
import cv2

In [37]:
from PIL import Image

In [68]:
for idx, img in enumerate(tqdm.tqdm(images_fake)):
  img = Image.open(img).convert("RGB").convert('L')
  img = img.resize((512,128))
  img.save(f'{fake_path}/{idx}.png')

100%|██████████| 1544/1544 [05:08<00:00,  5.00it/s]


In [39]:
images_real = glob.glob('/content/ScrabbleGAN/data/RIMES/validation/*')

In [40]:
real_path = 'temp_real'

In [69]:
for idx, img in enumerate(tqdm.tqdm(images_real)):
  img = Image.open(img).convert("RGB").convert('L')
  img = img.resize((512,128))
  img.save(f'{real_path}/{idx}.png')

100%|██████████| 1544/1544 [00:15<00:00, 98.29it/s]


In [70]:
!python -m pytorch_fid 'temp_real' 'temp_fake'

100% 31/31 [09:47<00:00, 18.96s/it]
100% 31/31 [09:55<00:00, 19.20s/it]
FID:  132.2877503050995
